# Использование модели "Мешка слов" для обработки текстов на естественном языке
Создадим рекомендательную систему, которая действует как вертикальная поисковая система. Система позволяет искать тексты из ограниченного количества документов, наиболее тесно связаны с заданной темой. Чтобы использовать возможности NLP, мы объединим методологию поиска с методами оценки семантического сходства.
Загрузим файл .csv и во фрейм данных.

In [1]:
import pandas as pd
import csv 
DATAPATH = 'Data/movies_metadata.csv'
df = pd.read_csv(DATAPATH)
# Remove null description
df = df[~df.isna()]
# Renaming the description column
df.rename(columns={'overview':'sentence'}, inplace=True)
# Sampling the first 5000 rows
df = df.iloc[:5000]
df.head()

C:\Users\acer\AppData\Local\Temp\ipykernel_15624\4103388425.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATAPATH)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,sentence,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Удаляем слишком короткие и слишком длинные предложения из нашего набора данных путем фильтрации с помощью MIN_WORDS и MAX_WORDS. 
Удаляем общеупотребительные слова (СТОП-СЛОВА), которые не помогают извлечь специфику данного предложения.

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 


def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers (punctuation, curly brackets etc).
        text (str): input text
    return (str): modified initial text
    """
    text = str(text).lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    Args:
      sentence (str)
      min_words (int)
      max_words (int)
      stopwords (set of string)
      lemmatize (boolean)
    returns:
      list of string
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    Args: 
      df (dataframe)
     returns:
      df
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS))
    return df
    
df = clean_sentences(df)

Cleaning sentences...


Функция, которая будет ранжировать лучшие рекомендации с учетом косинусного расстояния между векторами.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens ans return best mathes.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index



# Применение TF-IDF
Как только мы рассчитаем статистику модели Tf-IDF (веса слов с учетом их частоты в тексте, нормированной на частоту во всем корпусе документов), мы сможем сгенерировать вектор встраивания 22 180 измерений для каждого описания фильма. 
Эти функции хранятся в матрице функций tfidf_mat, где каждая строка представляет собой запись описания фильма, встроенную в вектор функций.
Когда мы получим запрос из пользовательского ввода, мы встроим его в одно и то же векторное пространство и сравним один за другим функцию предложения запроса embed_query с векторами предложений матрицы внедрения tfidf_mat .

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def get_recommendations_tfidf(sentence, tfidf_mat):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens_query = [str(tok) for tok in tokenizer(sentence)]
    embed_query = vectorizer.transform(tokens_query)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(embed_query, tfidf_mat)
    # Best cosine distance for each token independantly
    best_index = extract_best_indices(mat, topk=3)
    return best_index

# Adapt stop words
token_stop = tokenizer(' '.join(stopwords.words('english')), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(df['sentence'].values) # -> (num_sentences, num_vocabulary)

# Return best threee matches between query and dataset
test_sentence = 'a crime story with a beautiful woman' 
best_index = get_recommendations_tfidf(test_sentence, tfidf_mat)

display(df[['original_title', 'genres', 'sentence']].iloc[best_index])



D:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,original_title,genres,sentence
1060,Basic Instinct,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",A police detective is in charge of the investi...
793,Power 98,"[{'id': 28, 'name': 'Action'}, {'id': 9648, 'n...",A controversial talk show host becomes involve...
693,Mulholland Falls,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","In 1950s Los Angeles, a special crime squad of..."


Функция, которая находит лучший индекс из матрицы расстояний, берет среднее значение косинусного расстояния для каждого встроенного предложения и ранжирует результаты. Пока не обращайте внимания на аргумент маскировки, мы будем использовать его для другой модели.
Эта модель очень проста в использовании и может быть настроена с помощью всего нескольких строк кода, обучение также очень быстрое.

# Применение word2vec
Мы используем библиотеку gensim для обучения модели word2vec на наших данных корпуса. Сначала нам нужно предоставить словарь (список слов, которые мы хотим векторизовать), а затем обучить модель на нескольких эпохах. По умолчанию модель обучает CBOW.
Основным недостатком Word2Vec является то, что он не может создавать векторы для слов, которых изначально не было в используемом для обучения словаре. Вот почему нам нужно предоставить функцию is_word_in_model , которая удаляет слова из предложения запроса, которые не видны в обучающем наборе.
Мы используем встроенную функцию n_similarity для эффективного вычисления расстояния между запросом и предложениями набора данных.

In [12]:
from gensim.models.word2vec import Word2Vec

def is_word_in_model(word, model):
    """
    Check on individual words ``word`` that it exists in ``model``.
    """
    assert type(model).__name__ == 'KeyedVectors'
    is_in_vocab = word in model.key_to_index.keys()
    return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk=3):
    query_sentence = query_sentence.split()
    in_vocab_list, best_index = [], [0]*topk
    for w in query_sentence:
        # remove unseen words from query sentence
        if is_word_in_model(w, model.wv):
            in_vocab_list.append(w)
    # Retrieve the similarity between two words as a distance
    if len(in_vocab_list) > 0:
        sim_mat = np.zeros(len(dataset))  # TO DO
        for i, data_sentence in enumerate(dataset):
            if data_sentence:
                sim_sentence = model.wv.n_similarity(
                        in_vocab_list, data_sentence)
            else:
                sim_sentence = 0
            sim_mat[i] = np.array(sim_sentence)
        # Take the five highest norm
        best_index = np.argsort(sim_mat)[::-1][:topk]
    return best_index

# Create model
word2vec_model = Word2Vec(min_count=0, workers = 8, vector_size=300) 
# Prepare vocab
word2vec_model.build_vocab(df.tok_lem_sentence.values)
# Train
word2vec_model.train(df.tok_lem_sentence.values, total_examples=word2vec_model.corpus_count, epochs=30)

# Predict
best_index = predict_w2v(test_sentence, df['tok_lem_sentence'].values, word2vec_model)    
display(df[['original_title', 'genres', 'sentence']].iloc[best_index])

,original_title,genres,sentence
602,Heavy Metal,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",A glowing orb terrorizes a young girl with a c...
395,Homage,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",The young caretaker at the estate of a reclusi...
571,Andre,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",The true story of how a marine seal named Andr...


# Домашнее задание
- Разберитесь с приведенными примерами, поэкспериментируйте, меняя текст в примерах
- Разработайте программу – экспертную систему по подбору фильмов, для базы русскоязычных фильмов.

https://www.kaggle.com/datasets/alexandertesemnikov/kinopoisktop250russiandataset